Copyright 2018 The Dopamine Authors.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Dopamine: How to create and train a custom agent

This colab demonstrates how to create a variant of a provided agent (Example 1) and how to create a new agent from
scratch (Example 2).

Run all the cells below in order.

In [ ]:
# @title Install necessary packages.
!pip uninstall -y dopamine-rl
!git clone https://github.com/trotrem/dopamine.git
!pip install ./dopamine
!apt-get update && apt-get install cmake zlib1g-dev
!pip uninstall -y tensorflow
!pip install gast==0.2.2
!pip install absl-py atari-py gin-config gym opencv-python tensorflow==1.15
!pip uninstall -y seaborn
!pip install seaborn==0.9.1

#!pip install cmake
!pip install atari_py
#!pip install gin-config

In [ ]:
# @title Necessary imports and globals.
import numpy as np
import os
import sys
RANDOM_SEED = 1234
os.environ['PYTHONHASHSEED']=str(RANDOM_SEED)
import random
import tensorflow as tf
from dopamine.agents.rainbow import rainbow_agent
from dopamine.discrete_domains import run_experiment, gym_lib
from dopamine.colab import utils as colab_utils
import collections
from absl import flags
import gin.tf
import gym
from keras import backend as K
import math

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!gsutil -q -m cp -R gs://download-dopamine-rl/preprocessed-benchmarks/* /content/

gin.enter_interactive_mode()

Mounted at /content/gdrive


In [ ]:
# @title Load baseline data

BASE_PATH = '/content/gdrive/My Drive/proj/reinforcement/impl/'  # @param
#BASE_PATH = '/tmp/test/'  # @param
GAME = 'Asterix'  # @param
experimental_data = colab_utils.load_baselines('/content')

## Example 1: Train a modified version of DQN
Asterix is one of the standard agents provided with Dopamine.
The purpose of this example is to demonstrate how one can modify an existing agent. The modification
we are doing here (choosing actions randomly) is for illustrative purposes: it will clearly perform very
poorly.

In [ ]:
# @title batch convolution

def batch_convolution(A, B):
    output = z = tf.map_fn(lambda u: tf.map_fn(
        lambda v: tf.nn.conv1d(tf.expand_dims(tf.expand_dims(v[0],0),2),tf.expand_dims(tf.expand_dims(v[1],1),2),padding='SAME',stride=1), elems=[u[0],u[1]],dtype=tf.float32), elems=[A,B],dtype=tf.float32)

    return output[:,:,0,:,0]

In [ ]:
# @title random seed
tf.set_random_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

@gin.configurable
def create_seeded_env(environment_name=None, version='v0'):
  full_game_name = '{}-{}'.format(environment_name, version)
  env = gym.make(full_game_name)
  env = env.env

  env.seed(RANDOM_SEED)
  env.action_space.seed(RANDOM_SEED)

  env = gym_lib.GymPreprocessing(env)
  return env

In [ ]:
#@title Network { form-width: "10%" }
DRDNetworkType = collections.namedtuple(
    'DRD_network_type', ['q_values', 'logits', 'probabilities', 'sub_logits', 'sub_probabilities'])
gin.clear_config()
@gin.configurable
class DRDNetwork(tf.keras.Model):
  """The Distributional Reward Decomposition Network described in the paper"""

  def __init__(self, num_actions, num_atoms, support, N, vmax, name=None):
    """Creates the layers used calculating return distributions.
    Args:
      num_actions: int, number of actions.
      num_atoms: int, the number of buckets of the value function distribution.
      support: tf.linspace, the support of the Q-value distribution.
      name: str, used to crete scope for network parameters.
    """
    super(DRDNetwork, self).__init__(name=name)

    activation_fn = tf.keras.activations.relu
    self.num_actions = num_actions
    self.num_atoms = num_atoms
    self.support = support
    self.kernel_initializer = tf.keras.initializers.VarianceScaling(scale=1.0 / np.sqrt(3.0), mode='fan_in', distribution='uniform')
    # Defining layers.

    self.conv1 = tf.keras.layers.Conv2D(32, [8, 8], strides=4, padding='same', activation=activation_fn, kernel_initializer=self.kernel_initializer, name='Conv1')
    self.conv2 = tf.keras.layers.Conv2D(64, [4, 4], strides=2, padding='same', activation=activation_fn, kernel_initializer=self.kernel_initializer, name='Conv2')
    self.conv3 = tf.keras.layers.Conv2D(64, [3, 3], strides=1, padding='same', activation=activation_fn, kernel_initializer=self.kernel_initializer, name='Conv3')

    self.flatten = tf.keras.layers.Flatten()

    self.N = N
    self.denses = [tf.keras.layers.Dense(512, activation=activation_fn, kernel_initializer=self.kernel_initializer, name='fully_connected_1') for _ in range(self.N)]
    self.last_layers = [tf.keras.layers.Dense(num_actions * num_atoms, kernel_initializer=self.kernel_initializer, name='fully_connected_2') for _ in range(self.N)]

    self.support = support
    self.e = [tf.keras.layers.Input(tensor=tf.expand_dims(tf.keras.backend.one_hot(n, self.N), 0)) for n in range(self.N)]
    self.dense_phi = tf.keras.layers.Dense(7744, activation=activation_fn, name='phi_layer')
    print("network created")

  def call(self, state):
    """Creates the output tensor/op given the state tensor as input.
    See https://www.tensorflow.org/api_docs/python/tf/keras/Model for more
    information on this. Note that tf.keras.Model implements `call` which is
    wrapped by `__call__` function by tf.keras.Model.
    Args:
      state: Tensor, input tensor.
    Returns:
      collections.namedtuple, output ops (graph mode) or output tensors (eager).
    """
    # rainbow stuff
    print("call network")
    x = tf.cast(state, tf.float32)
    x = tf.div(x, 255.)
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.flatten(x)
    print("call rainbow stuff")
    sub_probs = []
    sub_logits = []


    #f = self.denses1[n](f)
    #f = self.denses2[n](f)

    # split for each sub-reward signal
    for n in range(self.N):
      phi_e = self.dense_phi(self.e[n])
      # element-wise multiplication with the one-hot layer to learn different distributions
      f = x
      f = f * phi_e
      print(f.shape)
      f = self.denses[n](f)

      f = self.last_layers[n](f)
      print(f.shape)
      print("call last layers")

      sub_logits.append(tf.reshape(f, [-1, self.num_actions, self.num_atoms]))

      print(sub_logits[-1].shape)
      # convolve sub distributions together
      sub_probs.append(tf.keras.activations.softmax(sub_logits[-1]))

    merged_probs = sub_probs[0]
    merged_logits = sub_logits[0]
    for n in range(1, self.N):
        merged_probs = batch_convolution(merged_probs, sub_probs[n])
        merged_logits = batch_convolution(merged_logits, sub_logits[n])
    print("merged_logits", merged_logits.shape)
    print("merged_probs", merged_probs.shape)

    q_values = tf.reduce_sum(self.support * merged_probs, axis=2)
    print("support", self.support)
    print("q_values", q_values.shape)
    print("network called")
    return DRDNetworkType(q_values, merged_logits, merged_probs, sub_logits, sub_probs)


In [ ]:
#@title Agent { form-width: "10%" }
LOG_PATH = os.path.join(BASE_PATH, 'rainbow_base', GAME)

from dopamine.discrete_domains import gym_lib
from dopamine.agents.dqn import dqn_agent

@gin.configurable
class DRDAgent(rainbow_agent.RainbowAgent):
  def __init__(self, sess, num_actions, N, disambiguation_factor,
               observation_shape=dqn_agent.NATURE_DQN_OBSERVATION_SHAPE,
               observation_dtype=dqn_agent.NATURE_DQN_DTYPE,
               stack_size=dqn_agent.NATURE_DQN_STACK_SIZE,
               network=DRDNetwork,
               num_atoms=51,
               vmax=10.,
               gamma=0.99,
               update_horizon=1,
               min_replay_history=20000,
               update_period=4,
               target_update_period=8000,
               epsilon_fn=dqn_agent.linearly_decaying_epsilon,
               epsilon_train=0.01,
               epsilon_eval=0.001,
               epsilon_decay_period=250000,
               replay_scheme='prioritized',
               tf_device='/cpu:*',
               use_staging=True,
               optimizer=tf.train.AdamOptimizer(
                   learning_rate=0.00025, epsilon=0.0003125),
               summary_writer=None,
               summary_writing_frequency=500):
    """This maintains all the rainbow default argument values."""
    self.N = N
    self.disambiguation_factor = disambiguation_factor
    num_atoms = 51 // self.N
    network = DRDNetwork
    self.vmax= vmax / self.N
    print("num_atoms",num_atoms)
    print("self.vmax",self.vmax)

    super(DRDAgent, self).__init__(sess, num_actions,
               observation_shape = observation_shape,
               observation_dtype = observation_dtype,
               stack_size = stack_size,
               network=network,
               num_atoms=num_atoms,
               vmax=self.vmax,
               gamma=gamma,
               update_horizon=update_horizon,
               min_replay_history=min_replay_history,
               update_period=update_period,
               target_update_period=target_update_period,
               epsilon_fn=epsilon_fn,
               epsilon_train=epsilon_train,
               epsilon_eval=epsilon_eval,
               epsilon_decay_period=epsilon_decay_period,
               replay_scheme=replay_scheme,
               tf_device=tf_device,
               use_staging=use_staging,
               optimizer=optimizer,
               summary_writer=summary_writer,
               summary_writing_frequency=summary_writing_frequency)


  def _create_network(self, name):
    """Builds a convolutional network that outputs Q-value distributions.
    Args:
      name: str, this name is passed to the tf.keras.Model and used to create
        variable scope under the hood by the tf.keras.Model.
    Returns:
      network: tf.keras.Model, the network instantiated by the Keras model.
    """
    print("create network")
    network = self.network(self.num_actions, self._num_atoms, self._support, self.N, self.vmax, name=name)
    #network = self.network(self.num_actions, self._num_atoms, self._support, name=name)
    return network

  def _build_train_op(self):
    """Builds a training op.
    Returns:
    train_op: An op performing one step of training from replay data.
    """
    print("train")
    # disambiguation
    # TODO batch
    print("output probs", self._replay_net_outputs.sub_probabilities[0].shape)
    print("output logits", self._replay_net_outputs.sub_logits[0].shape)
    if self.N > 1:
      argmax_probs = tf.argmax(tf.reduce_sum(self._support * self._replay_net_outputs.sub_probabilities, axis=-1), axis=-1)

      print("argmax_probs", argmax_probs.shape)
      N_indexes = tf.repeat(tf.range(self.N, dtype=tf.int64), self._replay.batch_size)
      batch_indexes = tf.repeat(tf.range(self._replay.batch_size, dtype=tf.int64), self.N)

      max_sub_probs = tf.reshape(tf.gather_nd(self._replay_net_outputs.sub_probabilities,tf.concat([tf.transpose([N_indexes, batch_indexes]), tf.reshape(argmax_probs, [self.N * self._replay.batch_size, 1])], axis=-1)),[self.N, self._replay.batch_size, self._num_atoms])
      print("max_sub_probs", max_sub_probs.shape)
      max_sub_logits = tf.reshape(tf.gather_nd(self._replay_net_outputs.sub_logits,tf.concat([tf.transpose([N_indexes, batch_indexes]), tf.reshape(argmax_probs, [self.N * self._replay.batch_size, 1])], axis=-1)),[self.N, self._replay.batch_size, self._num_atoms])
      kl_divs_sum = tf.add_n([tf.nn.softmax_cross_entropy_with_logits(
            labels=max_sub_probs[i], 
            logits=max_sub_logits[j]) for i in range(self.N) for j in range(self.N)  if i != j])
      print("kl_divs_sum", kl_divs_sum.shape)

    #kl_divs_sum = tf.Print(kl_divs_sum, [], message="train")

    target_distribution = tf.stop_gradient(self._build_target_distribution())
    print(target_distribution.shape)

    # size of indices: batch_size x 1.
    indices = tf.range(tf.shape(self._replay_net_outputs.logits)[0])[:, None]
    # size of reshaped_actions: batch_size x 2.
    reshaped_actions = tf.concat([indices, self._replay.actions[:, None]], 1)
    # For each element of the batch, fetch the logits for its selected apython on indexesction.
    chosen_action_logits = tf.gather_nd(self._replay_net_outputs.logits, reshaped_actions)
    print("chosen_logits", chosen_action_logits.shape)
    # C51 loss minus disambiguation
    c51_loss = tf.nn.softmax_cross_entropy_with_logits(
      labels=target_distribution, 
      logits=chosen_action_logits)
    print("c51_loss", c51_loss.shape)
    #print_op = tf.print("c51_loss:", c51_loss, "kl_divs loss:", self.disambiguation_factor * kl_divs_sum, output_stream=sys.stdout)
    if self.N > 1:
    #with tf.control_dependencies([print_op]):
      loss = c51_loss - self.disambiguation_factor * kl_divs_sum
    else:
      loss = c51_loss
    print("loss", loss.shape)

    if self._replay_scheme == 'prioritized':
      # The original prioritized experience replay uses a linear exponent
      # schedule 0.4 -> 1.0. Comparing the schedule to a fixed exponent of 0.5
      # on 5 games (Asterix, Pong, Q*Bert, Seaquest, Space Invaders) suggested
      # a fixed exponent actually performs better, except on Pong.
      probs = self._replay.transition['sampling_probabilities']
      loss_weights = 1.0 / tf.sqrt(probs + 1e-10)
      loss_weights /= tf.reduce_max(loss_weights)
  
      # Rainbow and prioritized replay are parametrized by an exponent alpha,
      # but in both cases it is set to 0.5 - for simplicity's sake we leave it
      # as is here, using the more direct tf.sqrt(). Taking the square root
      # "makes sense", as we are dealing with a squared loss.
      # Add a small nonzero value to the loss to avoid 0 priority items. While
      # technically this may be okay, setting all items to 0 priority will cause
      # troubles, and also result in 1.0 / 0.0 = NaN correction terms.
      update_priorities_op = self._replay.tf_set_priority(self._replay.indices, tf.sqrt(loss + 1e-10))
  
      # Weight the loss by the inverse priorities.
      loss = loss_weights * loss
    else:
      update_priorities_op = tf.no_op()
    print("okok")
    with tf.control_dependencies([update_priorities_op]):
      if self.summary_writer is not None:
        with tf.variable_scope('Losses'):
          tf.summary.scalar('CrossEntropyLoss', tf.reduce_mean(loss))
          # Schaul et al. reports a slightly different rule, where 1/N is also
          # exponentiated by beta. Not doing so seems more reasonable, and did not
          # impact performance in our experiments.

      return self.optimizer.minimize(tf.reduce_mean(loss)), loss

In [ ]:
#@title Config { form-width: "10%" }
def create_drd_agent(sess, environment, summary_writer=None):
  """The Runner class will expect a function of this type to create an agent."""
  return DRDAgent(sess, environment.action_space.n, N=2, disambiguation_factor=0.00001, vmax=10.)

drd_config = """
# Hyperparameters follow Hessel et al. (2018), except for sticky_actions,
# which was False (not using sticky actions) in the original paper.
import dopamine.agents.rainbow.rainbow_agent
import dopamine.discrete_domains.atari_lib
import dopamine.discrete_domains.run_experiment
import dopamine.replay_memory.prioritized_replay_buffer
import gin.tf.external_configurables

DRDAgent.num_atoms = 51
DRDAgent.vmax = 10.
DRDAgent.gamma = 0.99
DRDAgent.update_horizon = 3
DRDAgent.min_replay_history = 20000  # agent steps
DRDAgent.update_period = 4
DRDAgent.target_update_period = 8000  # agent steps
DRDAgent.epsilon_train = 0.01
DRDAgent.epsilon_eval = 0.001
DRDAgent.epsilon_decay_period = 250000  # agent steps
DRDAgent.replay_scheme = 'prioritized'
DRDAgent.tf_device = '/gpu:0'  # use '/cpu:*' for non-GPU version
DRDAgent.optimizer = @tf.train.AdamOptimizer()

# Note these parameters are different from C51's.
tf.train.AdamOptimizer.learning_rate = 0.0000625
tf.train.AdamOptimizer.epsilon = 0.00015

atari_lib.create_atari_environment.game_name = 'Asterix'
# Sticky actions with probability 0.25, as suggested by (Machado et al., 2017).
atari_lib.create_atari_environment.sticky_actions = True
create_agent.agent_name = 'DRD'
Runner.num_iterations = 100
Runner.training_steps = 250000  # agent steps
Runner.evaluation_steps = 125000  # agent steps
Runner.max_steps_per_episode = 27000  # agent steps

WrappedPrioritizedReplayBuffer.replay_capacity = 1000000
WrappedPrioritizedReplayBuffer.batch_size = 32
"""
gin.parse_config(drd_config, skip_unknown=False)

In [ ]:
#drd_runner = run_experiment.create_runner(LOG_PATH, schedule='continuous_train')
drd_runner = run_experiment.TrainRunner(LOG_PATH, create_drd_agent)
print('Will train agent, please be patient, may be a while...')
drd_runner.run_experiment()
print('Done training!')

In [ ]:
drd_data = colab_utils.read_experiment(LOG_PATH, verbose=True, summary_keys=['train_episode_returns'])
drd_data['agent'] = 'DRDAgent'
drd_data['run_number'] = 1
experimental_data[GAME] = experimental_data[GAME].merge(drd_data, how='outer')

In [ ]:
# @title Plot training results.
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16,8))
sns.tsplot(data=drd_data, time='iteration', unit='run_number',
           condition='agent', value='train_episode_returns', ax=ax)
plt.title(GAME)
plt.show()